# combine master files

## 필요한 모듈

이 프로젝트를 위해서는 아래의 모듈이 필요하다. 

> numpy, pandas, matplotlib, astropy, version_information

### 모듈 설치

1. 콘솔 창에서 모듈을 설치할 때는 아래와 같은 형식으로 입력하면 된다.

>pip install module_name==version

>conda install module_name==version

2. 주피터 노트북(코랩 포함)에 설치 할 때는 아래의 셀을 실행해서 실행되지 않은 모듈을 설치할 수 있다. (pip 기준) 만약 아나콘다 환경을 사용한다면 7행을 콘다 설치 명령어에 맞게 수정하면 된다.

In [6]:
# Install a pip package in the current Jupyter kernel
import importlib, sys, subprocess
print(f"sys.executable: {sys.executable}")
#!{sys.executable} -m pip install numpy==1.22 pillow==8.4 matplotlib==3.2

packages = "numpy, pandas, matplotlib, scipy, astropy, photutils, ccdproc, version_information" # required modules
pkgs = packages.split(", ")
for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
        print(f"**** {pkg} module is now installed.")
    else: 
        print(f"******** {pkg} module is already installed.")
%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")


sys.executable: /home/guitar79/anaconda3/envs/astro_Python_env/bin/python
******** numpy module is already installed.
******** pandas module is already installed.
******** matplotlib module is already installed.
******** scipy module is already installed.
******** astropy module is already installed.
******** photutils module is already installed.
******** ccdproc module is already installed.
******** version_information module is already installed.
The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information
This notebook was generated at 2024-06-01 09:49:07 (KST = GMT+0900) 
0 Python     3.11.5 64bit [GCC 11.2.0]
1 IPython    8.20.0
2 OS         Linux 5.15.0 107 generic x86_64 with glibc2.31
3 numpy      1.26.4
4 pandas     2.2.1
5 matplotlib 3.8.4
6 scipy      1.12.0
7 astropy    5.3.4
8 photutils  1.9.0
9 ccdproc    2.4.2
10 version_information 1.0.4


### 모듈 버전 확인

아래 셀을 실행하면 이 노트북을 실행한 파이썬 및 관련 모듈의 버전을 확인할 수 있다.

### import modules

In [7]:
from glob import glob
from pathlib import Path
import os
import shutil
import numpy as np
import astropy.units as u
from astropy.stats import sigma_clip
from ccdproc import combine, ccd_process, CCDData

import ysfitsutilpy as yfu
import ysphotutilpy as ypu
#import ysvisutilpy as yvu

import _astro_utilities
import _Python_utilities

In [8]:
#%%
BASEDIR = Path("/mnt/Rdata/OBS_data") 
DOINGDIR = Path(BASEDIR/ "2024-Spectra" / "TEC140_ASI183MMPro_-_1bin")

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(DOINGDIR))
DOINGDIRs = sorted([x for x in DOINGDIR.iterdir() if x.is_dir()])
print ("DOINGDIRs: ", format(DOINGDIRs))
print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

MASTERDIR = [x for x in DOINGDIRs if "CAL-BDF" in str(x)]
MASTERDIR = MASTERDIR[0]/ _astro_utilities.master_dir
print ("MASTERDIR: ", format(MASTERDIR))

DOINGDIRs = sorted([x for x in DOINGDIRs if "_LIGHT_" in str(x)])
print ("DOINGDIRs: ", format(DOINGDIRs))
print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))
#######################################################

DOINGDIRs:  [PosixPath('/mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/-_CAL-BDF_-_2024-05-14_-_TEC140_ASI183MMPro_-_1bin'), PosixPath('/mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-CAL_LIGHT_-_2024-05-14_-_TEC140_ASI183MMPro_-_1bin'), PosixPath('/mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP24608-CAPELA_LIGHT_-_2024-04-26_-_TEC140_ASI183MMPro_-_1bin'), PosixPath('/mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP27989-BETELGEUSE_LIGHT_-_2024-05-01_-_TEC140_ASI183MMPro_-_1bin'), PosixPath('/mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP31681-ALHENA_LIGHT_-_2024-05-01_-_TEC140_ASI183MMPro_-_1bin'), PosixPath('/mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP32246-MEBSUTA_LIGHT_-_2024-05-01_-_TEC140_ASI183MMPro_-_1bin'), PosixPath('/mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP32349-SIRIUS_LIGHT_-_2024-04-26_-_TEC140_ASI183MMPro_-_1bin'), PosixPath('/mnt/Rdata/OB

In [9]:
# import shutil
# MASTERDIR = Path(DOINGDIRs[0]/_astro_utilities.master_dir)

In [11]:
for DOINGDIR in DOINGDIRs[:] :
    DOINGDIR = Path(DOINGDIR)
    print("DOINGDIR", DOINGDIR)
    fits_in_dir = sorted(list(DOINGDIR.glob('*.fit*')))
    #print("fits_in_dir", fits_in_dir)
    print("len(fits_in_dir)", len(fits_in_dir))

    if len(fits_in_dir) == 0 :
        print(f"There is no fits fils in {DOINGDIR}")
        pass
    else : 
        print(f"Starting: {str(DOINGDIR.parts[-1])}")
    
        MASTERDIR = DOINGDIR / _astro_utilities.master_dir
        REDUCEDDIR = DOINGDIR / _astro_utilities.reduced_dir

        if not MASTERDIR.exists():
            #shutil.rmtree(MASTERDIR)
            try: 
                shutil.copytree(MASTERDIR, MASTERDIR, dirs_exist_ok=True)
            except Exception as err:
                print(err)

DOINGDIR /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-CAL_LIGHT_-_2024-05-14_-_TEC140_ASI183MMPro_-_1bin
len(fits_in_dir) 69
Starting: SPTR-CAL_LIGHT_-_2024-05-14_-_TEC140_ASI183MMPro_-_1bin
DOINGDIR /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP24608-CAPELA_LIGHT_-_2024-04-26_-_TEC140_ASI183MMPro_-_1bin
len(fits_in_dir) 62
Starting: SPTR-HIP24608-CAPELA_LIGHT_-_2024-04-26_-_TEC140_ASI183MMPro_-_1bin
DOINGDIR /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP27989-BETELGEUSE_LIGHT_-_2024-05-01_-_TEC140_ASI183MMPro_-_1bin
len(fits_in_dir) 56
Starting: SPTR-HIP27989-BETELGEUSE_LIGHT_-_2024-05-01_-_TEC140_ASI183MMPro_-_1bin
DOINGDIR /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP31681-ALHENA_LIGHT_-_2024-05-01_-_TEC140_ASI183MMPro_-_1bin
len(fits_in_dir) 9
Starting: SPTR-HIP31681-ALHENA_LIGHT_-_2024-05-01_-_TEC140_ASI183MMPro_-_1bin
DOINGDIR /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP32246

In [5]:
for DOINGDIR in DOINGDIRs[1:] :
    DOINGDIR = Path(DOINGDIR)
    print("DOINGDIR", DOINGDIR)
    fits_in_dir = sorted(list(DOINGDIR.glob('*.fit*')))
    #print("fits_in_dir", fits_in_dir)
    print("len(fits_in_dir)", len(fits_in_dir))

    if len(fits_in_dir) == 0 :
        print(f"There is no fits fils in {DOINGDIR}")
        pass
    else : 
        print(f"Starting: {str(DOINGDIR.parts[-1])}")
    
        MASTERDIR = DOINGDIR / _astro_utilities.master_dir
        REDUCEDDIR = DOINGDIR / _astro_utilities.reduced_dir

        if not MASTERDIR.exists():
            #shutil.rmtree(MASTERDIR)
            try: 
                shutil.copytree(MASTERDIR, MASTERDIR, dirs_exist_ok=True)
            except Exception as err:
                print(err)

        if not REDUCEDDIR.exists():
            os.makedirs(str(REDUCEDDIR))
            print("{} is created...".format(str(REDUCEDDIR)))

        summary = yfu.make_summary(DOINGDIR/"*.fit*")
        #print(summary)
        print("len(summary):", len(summary))
        print("summary:", summary)
        #print(summary["file"][0])

        df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
        df_light = df_light.reset_index(drop=True)

        for _, row in df_light.iterrows():

            fpath = Path(row["file"])
            ccd = yfu.load_ccd(fpath)
            filt = ccd.header["FILTER"]
            expt = ccd.header["EXPTIME"]
            try : 
                red = yfu.ccdred(
                    ccd,
                    output=Path(f"{REDUCEDDIR/ fpath.name}"),
                    mdarkpath=str(MASTERDIR / "master_dark_{:.0f}sec.fits".format(expt)),
                    mflatpath=str(MASTERDIR / "master_flat_{}_norm.fits".format(filt.upper())),
                    # flat_norm_value=1,  # 1 = skip normalization, None = normalize by mean
                    overwrite=True
                )
            except Exception as err :
                print("X"*60)
                #_Python_utilities.write_log(err_log_file, err)

DOINGDIR /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP24608-CAPELA_LIGHT_-_2024-04-26_-_TEC140_ASI183MMPro_-_1bin
len(fits_in_dir) 62
Starting: SPTR-HIP24608-CAPELA_LIGHT_-_2024-04-26_-_TEC140_ASI183MMPro_-_1bin
/mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP24608-CAPELA_LIGHT_-_2024-04-26_-_TEC140_ASI183MMPro_-_1bin/reduced is created...
All 48 keywords (guessed from /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP24608-CAPELA_LIGHT_-_2024-04-26_-_TEC140_ASI183MMPro_-_1bin/SPTR-HIP24608-CAPELA_LIGHT__2024-04-26-12-11-33_5sec_TEC140_ASI183MMPro_-20c_1bin.fit) will be loaded.
len(summary): 62
summary:                                                  file  filesize  SIMPLE  \
0   /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183...  40368960    True   
1   /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183...  40368960    True   
2   /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183...  40368960    True   
3   /mnt/Rdata/OBS_data/2024-Sp

Set MJD-AVG to 60426.508061 from DATE-AVG'. [astropy.wcs.wcs]
Set MJD-AVG to 60430.428827 from DATE-AVG'. [astropy.wcs.wcs]
Set MJD-AVG to 60444.266387 from DATE-AVG'. [astropy.wcs.wcs]


HISTORY  [yfu.darkcor] Dark subtracted (DARKFRM = /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP24608-CAPELA_LIGHT_-_2024-04-26_-_TEC140_ASI183MMPro_-_1bin/master_files_ys/master_dark_5sec.fits)
HISTORY  ..................................(dt = 0.031 s) 2024-05-31T23:21:47.903
HISTORY  [yfu.flatcor] Flat pixels with `value < flat_mask = 0` are replaced by `flat_fill = 1`.
HISTORY  .................................................2024-05-31T23:21:47.928
HISTORY  [yfu.flatcor] Flat corrected (FLATFRM = /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP24608-CAPELA_LIGHT_-_2024-04-26_-_TEC140_ASI183MMPro_-_1bin/master_files_ys/master_flat__norm.fits)
HISTORY  ..................................(dt = 0.039 s) 2024-05-31T23:21:47.947
Writing FITS to /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP24608-CAPELA_LIGHT_-_2024-04-26_-_TEC140_ASI183MMPro_-_1bin/reduced/SPTR-HIP24608-CAPELA_LIGHT__2024-04-26-12-11-33_5sec_TEC140_ASI183MMPro_-2

Set MJD-AVG to 60426.508151 from DATE-AVG'. [astropy.wcs.wcs]


HISTORY  [yfu.darkcor] Dark subtracted (DARKFRM = /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP24608-CAPELA_LIGHT_-_2024-04-26_-_TEC140_ASI183MMPro_-_1bin/master_files_ys/master_dark_5sec.fits)
HISTORY  ..................................(dt = 0.031 s) 2024-05-31T23:21:48.384
HISTORY  [yfu.flatcor] Flat pixels with `value < flat_mask = 0` are replaced by `flat_fill = 1`.
HISTORY  .................................................2024-05-31T23:21:48.405
HISTORY  [yfu.flatcor] Flat corrected (FLATFRM = /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP24608-CAPELA_LIGHT_-_2024-04-26_-_TEC140_ASI183MMPro_-_1bin/master_files_ys/master_flat__norm.fits)
HISTORY  ..................................(dt = 0.039 s) 2024-05-31T23:21:48.424
Writing FITS to /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP24608-CAPELA_LIGHT_-_2024-04-26_-_TEC140_ASI183MMPro_-_1bin/reduced/SPTR-HIP24608-CAPELA_LIGHT__2024-04-26-12-11-41_5sec_TEC140_ASI183MMPro_-2

...

Set MJD-AVG to 60443.655669 from DATE-AVG'. [astropy.wcs.wcs]


HISTORY  [yfu.darkcor] Dark subtracted (DARKFRM = /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP59774-MEGREZ_LIGHT_-_2024-05-13_-_TEC140_ASI183MMPro_-_1bin/master_files_ys/master_dark_60sec.fits)
HISTORY  ..................................(dt = 0.062 s) 2024-05-31T23:37:45.167
HISTORY  [yfu.flatcor] Flat pixels with `value < flat_mask = 0` are replaced by `flat_fill = 1`.
HISTORY  .................................................2024-05-31T23:37:45.187
HISTORY  [yfu.flatcor] Flat corrected (FLATFRM = /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP59774-MEGREZ_LIGHT_-_2024-05-13_-_TEC140_ASI183MMPro_-_1bin/master_files_ys/master_flat__norm.fits)
HISTORY  ..................................(dt = 0.044 s) 2024-05-31T23:37:45.212
Writing FITS to /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP59774-MEGREZ_LIGHT_-_2024-05-13_-_TEC140_ASI183MMPro_-_1bin/reduced/SPTR-HIP59774-MEGREZ_LIGHT__2024-05-13-15-43-39_60sec_TEC140_ASI183MMPro_

Set MJD-AVG to 60443.656374 from DATE-AVG'. [astropy.wcs.wcs]


HISTORY  [yfu.darkcor] Dark subtracted (DARKFRM = /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP59774-MEGREZ_LIGHT_-_2024-05-13_-_TEC140_ASI183MMPro_-_1bin/master_files_ys/master_dark_60sec.fits)
HISTORY  ..................................(dt = 0.031 s) 2024-05-31T23:37:46.105
HISTORY  [yfu.flatcor] Flat pixels with `value < flat_mask = 0` are replaced by `flat_fill = 1`.
HISTORY  .................................................2024-05-31T23:37:46.125
HISTORY  [yfu.flatcor] Flat corrected (FLATFRM = /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP59774-MEGREZ_LIGHT_-_2024-05-13_-_TEC140_ASI183MMPro_-_1bin/master_files_ys/master_flat__norm.fits)
HISTORY  ..................................(dt = 0.038 s) 2024-05-31T23:37:46.145
Writing FITS to /mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP59774-MEGREZ_LIGHT_-_2024-05-13_-_TEC140_ASI183MMPro_-_1bin/reduced/SPTR-HIP59774-MEGREZ_LIGHT__2024-05-13-15-44-40_60sec_TEC140_ASI183MMPro_

Error: [('/mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/-_CAL-BDF_-_2024-05-14_-_TEC140_ASI183MMPro_-_1bin/master_files_ys/master_bias.fits', '/mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/SPTR-HIP61941-PORRIMA_LIGHT_-_2024-05-02_-_TEC140_ASI183MMPro_-_1bin/master_files_ys/master_bias.fits', "[Errno 2] No such file or directory: '/mnt/Rdata/OBS_data/2024-Spectra/TEC140_ASI183MMPro_-_1bin/-_CAL-BDF_-_2024-05-14_-_TEC140_ASI183MMPro_-_1bin/master_files_ys/master_bias.fits'")]